<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#АВС-анализ" data-toc-modified-id="АВС-анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>АВС-анализ</a></span><ul class="toc-item"><li><span><a href="#Одномерный-(по-количеству)" data-toc-modified-id="Одномерный-(по-количеству)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Одномерный (по количеству)</a></span></li><li><span><a href="#Многомерный" data-toc-modified-id="Многомерный-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Многомерный</a></span></li><li><span><a href="#Визуализация-групп" data-toc-modified-id="Визуализация-групп-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Визуализация групп</a></span></li></ul></li><li><span><a href="#Выводы-и-рекомендации" data-toc-modified-id="Выводы-и-рекомендации-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы и рекомендации</a></span><ul class="toc-item"><li><span><a href="#Группа-A**" data-toc-modified-id="Группа-A**-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Группа A**</a></span></li><li><span><a href="#группа-B**" data-toc-modified-id="группа-B**-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>группа B**</a></span></li><li><span><a href="#группа-C**" data-toc-modified-id="группа-C**-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>группа C**</a></span></li></ul></li></ul></div>

__Импорты и функции__

In [42]:
import pandas as pd
import numpy as np
import plotly.express as px

#dataset settings
pd.set_option('display.max_columns', 100) #макс кол-во колонок в выводимых данных 
pd.set_option('display.max_rows', 30) #макс кол-во строк в выводимых данных
pd.set_option('display.width', 60) #макс кол-во символов в строке

#warnings and other settings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None

Для многомерного `АВС-анализа` напишем и будем использовать эту функцию: 

In [43]:
def abc_multi_func(dataset, columns, a_limit=0.8, b_limit=0.95): 
    '''
    Функция принимает на вход pandas DataFrame и список столбцов, 
    а также опциональные параметры, устанавливающие границы для категорий. 
    
    Функция возвращает pandas DataFrame с указанием категории АВС для каждого 
    из столбцов, по которому проводился анализ. 
    
    Для каждого столбца выполняет следующие операции:
    1. Создает новый столбец "relative_{column}", содержащий относительные 
    значения каждой ячейки в столбце относительно суммы всех значений в этом столбце.
    2. Сортирует датасет по этому новому столбцу по убыванию.
    3. Создает новый столбец "cumsum_{column}", содержащий накопленные суммы значений 
    из столбца "relative_{column}".
    4. Создает новый столбец "abc_{column}", присваивая значения 'A', 'B' или 'C' 
    в зависимости от кумулятивной суммы в столбце "cumsum_{column}" и 
    пороговых значений a_limit и b_limit.
    5. Формирует итоговый индекс ABC, состоящий из индексов по каждому направлению анализа
    
    Args:
    - dataset: pandas DataFrame, входной датасет
    - columns: list, список столбцов, для которых необходимо выполнить операции
    - a_limit: float, опциональный параметр, пороговое значение для класса A (по умолчанию 0.8)
    - b_limit: float, опциональный параметр, пороговое значение для класса B (по умолчанию 0.95)
    
    Returns:
    - pandas DataFrame 
    
    '''
    
    dataset['ABC'] = ''
    
    for column in columns: 
        
        dataset[f'relative_{column}'] = dataset[column] / dataset[column].sum()
        
        dataset = dataset.sort_values(by=f'relative_{column}', ascending=False)
        
        dataset[f'cumsum_{column}'] = dataset[f'relative_{column}'].cumsum()
        
        dataset[f'abc_{column}'] = np.where(
            dataset[f'cumsum_{column}'] < a_limit, 'A', 
            np.where(
                dataset[f'cumsum_{column}'] < b_limit, 'B', 'C'
            )
        )
        
        dataset['ABC'] += dataset[f'abc_{column}']
    
    return dataset

Зададим порог отнесения к группам. 

In [44]:
A_LIMIT = 0.80
B_LIMIT = 0.95

## Подготовка данных

In [45]:
df = pd.read_excel('spb_cat_sales.xlsx')
df.head()

,extgroup_name,subgroup_name,article_name,total_quantity,gross_sales
0,Catalog SPB,Винная карта,"""Маэстро"" Саперави Бадагони п/сл 0,75",57,136885
1,Catalog SPB,Винная карта,"""Парфюм д'Эте Шато Мухрани 0.75",9,27435
2,Catalog SPB,Чайный мастер,1000 мл Бисмарк,163,112470
3,Catalog SPB,Чайный мастер,1000 мл Иван Чай,5,2450
4,Catalog SPB,Чайный мастер,1000 мл Легенда Осени,39,29214


In [46]:
df.isna().sum()

extgroup_name     0
subgroup_name     0
article_name      0
total_quantity    0
gross_sales       0
dtype: int64

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   extgroup_name   694 non-null    object
 1   subgroup_name   694 non-null    object
 2   article_name    694 non-null    object
 3   total_quantity  694 non-null    int64 
 4   gross_sales     694 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 27.2+ KB


In [48]:
df.subgroup_name.unique()

array(['Винная карта', 'Чайный мастер', 'хц закуски', 'Хинкали и соуса',
       'Спец предложение Кухня', 'БарНАЯ КАРТА', 'Выпечка',
       'Вина по бокалам', 'Спец предложение Бар', 'Коктейли',
       'хц десерты', 'Банкет Кухня', 'гц мангал', 'Без скидки',
       'Завтраки', 'РАСПРОДАЖА 50%', 'Бар Распродажа 30%',
       'гц основные блюда и гарниры', 'гц закуски', 'Банкет Бар',
       'гц супа', 'Компоты', 'Базар/Магазин', 'Наборы', 'Праздник',
       'хц салаты'], dtype=object)

Нас интересуют только позиции основного меню, поэтому отберем их. Позиции бара и др.подкатегорий логично анализировать отдельно. 

In [49]:
food = ['Хинкали и соуса','Выпечка','гц основные блюда и гарниры','гц закуски',
'хц салаты','хц закуски','гц мангал','гц супа','Завтраки']

In [50]:
df.loc[df['subgroup_name'].apply(lambda x: x in food)]['article_name'].unique()

array(['4 пхали на лаваше с баклажаном', '4 пхали на лаваше с грибами',
       '4 пхали на лаваше со свеклой', '4 пхали на лаваше со шпинатом',
       '4шт Хинкали с бараниной и говядиной',
       '4шт Хинкали с говядиной и свининой', '4шт Хинкали с грибами',
       '4шт Хинкали с мраморной говядиной', '4шт Хинкали с сыром',
       'Аджабсандал', 'Аджарик', 'Аджика красная',
       'Ассорти закусок "Кутаиси"', 'Ассорти пхали на лаваше',
       'Ассорти рулетиков с сыром надуги с мятой', 'Ассорти сыров',
       'Баже', 'Баранья лопатка на кости', 'Бастурма 30 гр',
       'Ветчина топпинг', 'Вишневый соус 50гр', 'Грузинский лаваш пури',
       'Дет. Картофель фри', 'Дет. Картофельное пюре',
       'Дет. Куриная котлета', 'Дет. Куриный бульон с алфавитом',
       'Дет. Куриный шашлычок', 'Дет. Макарошки',
       'Дет. Наггетсы куриные в кляре', 'Дет. Наггетсы рыбные в кляре',
       'Дет. Овощной салат', 'Дет. Овощные палочки', 'Дет. Омлет Цыпа',
       'Дет. Пиццури с сыром и ветчиной', 

In [51]:
df_food = df.loc[df['subgroup_name'].apply(
    lambda x: x in food)].reset_index()[['article_name', 'total_quantity', 'gross_sales']]

In [52]:
df_food.head()

,article_name,total_quantity,gross_sales
0,4 пхали на лаваше с баклажаном,1214,315640
1,4 пхали на лаваше с грибами,581,151060
2,4 пхали на лаваше со свеклой,379,98540
3,4 пхали на лаваше со шпинатом,365,94900
4,4шт Хинкали с бараниной и говядиной,95851,26315940


Добавим данные о коэффициенте прибыльности товаров, выгруженные из другой базы.

In [53]:
margin = pd.read_excel('margin_coefficients.xlsx')
margin.head()

,article_name,profit_coef
0,2 шт Сосиска в тесте,1.759945
1,4 пхали на лаваше с баклажаном,3.442841
2,4 пхали на лаваше с грибами,2.767872
3,4 пхали на лаваше со свеклой,6.403552
4,4 пхали на лаваше со шпинатом,3.085769


In [54]:
df_food = df_food.merge(margin, 
          how='left',
          left_on='article_name', 
          right_on='article_name')
          #how='left')#, 
          #lsuffix='', 
          #rsuffix='', 
          #sort=False

In [55]:
df_food['profit_coef'] = df_food['profit_coef'].fillna(0.72)

In [56]:
# df_food['profit'] = (
#     (df_food['gross_sales'] / df_food['total_quantity']) - (
#         ((df_food['gross_sales'] / df_food['total_quantity'])
#         ) / (df_food['profit_coef'] + 1))) * df_food['total_quantity']

# col_list = list(df_food.columns)
# col_list.remove('profit_coef')
# df_food = df_food[col_list]

In [57]:
df_food.head()

,article_name,total_quantity,gross_sales,profit_coef
0,4 пхали на лаваше с баклажаном,1214,315640,3.442841
1,4 пхали на лаваше с грибами,581,151060,2.767872
2,4 пхали на лаваше со свеклой,379,98540,6.403552
3,4 пхали на лаваше со шпинатом,365,94900,3.085769
4,4шт Хинкали с бараниной и говядиной,95851,26315940,1.999417


In [58]:
df_food.groupby(by=['article_name']).sum().reset_index().sort_values(
    by='total_quantity', ascending=False).head(10)

,article_name,total_quantity,gross_sales,profit_coef
134,Хинкали с говядиной и свининой,282235,19265278,2.928124
133,Хинкали с бараниной и говядиной,247748,16906222,2.058434
136,Хинкали с мраморной говядиной,163556,11174301,3.039062
137,Хинкали с сыром сулугуни,135271,9231675,1.380786
5,4шт Хинкали с говядиной и свининой,124508,34198160,3.034428
4,4шт Хинкали с бараниной и говядиной,95851,26315940,1.999417
10,Аджарик,86839,18078106,1.867914
7,4шт Хинкали с мраморной говядиной,68674,18814240,2.804297
135,Хинкали с грибами,67900,4630978,2.671634
8,4шт Хинкали с сыром,64193,17614980,1.295483


Кратко проверили данные, подготовили датасет для построения анализа. 

## АВС-анализ

### Одномерный (по количеству)

In [59]:
df1 = df_food.copy()

In [60]:
df1['relative_qty'] = df1['total_quantity'] / df1['total_quantity'].sum()
df1 = df1.sort_values(by='relative_qty', ascending=False)
df1['cumsum_qty'] = df1['relative_qty'].cumsum()

df1['abc_qty'] = np.where(
    df1['cumsum_qty'] < A_LIMIT, 'A', 
    np.where(
        df1['cumsum_qty'] < B_LIMIT, 'B', 'C'
    )
)

In [61]:
df1

,article_name,total_quantity,gross_sales,profit_coef,relative_qty,cumsum_qty,abc_qty
134,Хинкали с говядиной и свининой,282235,19265278,2.928124,0.117014,0.117014,A
133,Хинкали с бараниной и говядиной,247748,16906222,2.058434,0.102715,0.219729,A
136,Хинкали с мраморной говядиной,163556,11174301,3.039062,0.067810,0.287539,A
137,Хинкали с сыром сулугуни,135271,9231675,1.380786,0.056083,0.343622,A
5,4шт Хинкали с говядиной и свининой,124508,34198160,3.034428,0.051621,0.395242,A
...,...,...,...,...,...,...,...
157,Язык отварной 50 гр,15,4500,1.751183,0.000006,0.999983,C
93,Пхали со свеклой,14,4495,7.498883,0.000006,0.999988,C
112,Суджук из говядины 30 гр,13,2990,2.028962,0.000005,0.999994,C
39,Джонджоли 50 гр,12,4375,3.045145,0.000005,0.999999,C


Одномерный `АВС-анализ` не является достаточным для того, чтобы по нему делать выводы о товаре, т.к. товары, относящиеся к группе `С` по количеству, например, могут быть очень дорогими/маржинальными и приносить много выручки/прибыли. 

Или, если мы, основываясь на одномерном анализе, возьмем какой-либо товар в группе `А` и поднимем на него цену с целью увеличить выручку, мы можем не получить ожидаемый результат, т.к. мы не знаем, регулярный на эту позицию товара был спрос или это какой-то разовый спрос после акции, например (без `XYZ-анализа`)

В данном конкретном случае мы знаем, что позиции из группы `А` не являются позициями разового спроса, но тем не менее, продолжим анализ, чтобы наши выводы не были поверхностными.

### Многомерный

In [62]:
df2 = df_food.copy()

In [63]:
cols = list(df2.columns)
cols.remove('article_name')
cols

['total_quantity', 'gross_sales', 'profit_coef']

In [64]:
df_food_abc = abc_multi_func(df2, cols, A_LIMIT, B_LIMIT)

In [65]:
df_food_abc.head()

,article_name,total_quantity,gross_sales,profit_coef,ABC,relative_total_quantity,cumsum_total_quantity,abc_total_quantity,relative_gross_sales,cumsum_gross_sales,abc_gross_sales,relative_profit_coef,cumsum_profit_coef,abc_profit_coef
50,Каша овсяная на воде,52,11440,9.331748,CCA,0.000022,0.999800,C,0.000023,0.999736,C,0.022668,0.022668,A
21,Грузинский лаваш пури,24921,2243064,9.142169,ABA,0.010332,0.719573,A,0.004455,0.902094,B,0.022208,0.044876,A
93,Пхали со свеклой,14,4495,7.498883,CCA,0.000006,0.999988,C,0.000009,0.999903,C,0.018216,0.063092,A
64,Мини-хинкали с аджабсандалом,374,145899,7.278672,CCA,0.000155,0.997621,C,0.000290,0.996733,C,0.017681,0.080773,A
2,4 пхали на лаваше со свеклой,379,98540,6.403552,CCA,0.000157,0.997466,C,0.000196,0.998172,C,0.015555,0.096329,A


Взглянем на "лучшие" и "худшие" позиции

In [66]:
df_food_abc[df_food_abc['ABC']=='AAA'].sort_values(by='total_quantity', ascending=False).head()

,article_name,total_quantity,gross_sales,profit_coef,ABC,relative_total_quantity,cumsum_total_quantity,abc_total_quantity,relative_gross_sales,cumsum_gross_sales,abc_gross_sales,relative_profit_coef,cumsum_profit_coef,abc_profit_coef
134,Хинкали с говядиной и свининой,282235,19265278,2.928124,AAA,0.117014,0.117014,A,0.038259,0.199984,A,0.007113,0.503663,A
133,Хинкали с бараниной и говядиной,247748,16906222,2.058434,AAA,0.102715,0.219729,A,0.033574,0.341805,A,0.005000,0.765587,A
136,Хинкали с мраморной говядиной,163556,11174301,3.039062,AAA,0.067810,0.287539,A,0.022191,0.453999,A,0.007382,0.445346,A
5,4шт Хинкали с говядиной и свининой,124508,34198160,3.034428,AAA,0.051621,0.395242,A,0.067915,0.067915,A,0.007371,0.452718,A
4,4шт Хинкали с бараниной и говядиной,95851,26315940,1.999417,AAA,0.039739,0.434982,A,0.052261,0.120176,A,0.004857,0.790233,A


In [67]:
df_food_abc[df_food_abc['ABC']=='CCC'].sort_values(by='total_quantity', ascending=False).tail()

,article_name,total_quantity,gross_sales,profit_coef,ABC,relative_total_quantity,cumsum_total_quantity,abc_total_quantity,relative_gross_sales,cumsum_gross_sales,abc_gross_sales,relative_profit_coef,cumsum_profit_coef,abc_profit_coef
76,Орех грецкий топпинг,97,4850,0.425793,CCC,0.000040,0.999540,C,0.000010,0.999885,C,0.001034,0.999087,C
18,Бастурма 30 гр,56,14430,0.744414,CCC,0.000023,0.999757,C,0.000029,0.999713,C,0.001808,0.988408,C
121,Сыр чечил копченый 30 гр,41,2460,1.247762,CCC,0.000017,0.999911,C,0.000005,0.999980,C,0.003031,0.958067,C
69,Огурцы маринованные 50 гр,41,2050,0.970128,CCC,0.000017,0.999894,C,0.000004,0.999997,C,0.002357,0.974018,C
110,Соус табаско 10 гр,23,2070,0.485028,CCC,0.000010,0.999947,C,0.000004,0.999989,C,0.001178,0.998053,C


### Визуализация групп

In [68]:
abc_groups = df_food_abc.groupby(by=['ABC'])['article_name'].count().reset_index().sort_values(by=['ABC'])

In [69]:
abc_groups

,ABC,article_name
0,AAA,16
1,AAB,6
2,AAC,2
3,ABA,5
4,ABB,1
5,ACA,1
6,BAA,4
7,BAB,9
8,BAC,2
9,BBA,9


In [70]:
fig = px.treemap(abc_groups, path=['ABC'], values='article_name')
fig.show()

По количеству позиций наиболее многочисленная группа `CC*` (86 позиций - половина из всех), в чем для нас нет ничего неожиданного, т.к. она содержит много топпингов, добавок и соусов, без которых блюда могут стать весьма однообразными, особенно те, что составляют "топ" наших позиций - `AAA` - все хинкали, однако, эти позиции не являются основными.  

Однако она содержит и позиции основного меню, по которым необходимо предпринимать действия для того, чтобы "перетянуть" их в группу `B`. Кроме того также и топпинги, и соусы возможно продавать больше, это повысит средний чек.

## Выводы и рекомендации

Основная задача `ABC-анализа` не в том, чтобы вывести те позиции, которые у нас "не в топе", а в том, чтобы понять, какие позиции и каким образом мы можем тянуть по этим группам вверх. 

! `ABC-анализ` блюд необходимо делать совместно с `XYZ-анализом`, и рекомендации должны базироваться на результатах двух анализов.

### Группа A**

Группа A — самые важные позиции в меню, приносящие основную часть дохода, поэтому данные блюда настоятельно не рекомендуется убирать из меню. Данные позиции пользуются популярностью у гостей, вероятно многие гости посещают заведение именно с целью выбора блюд этой группы.

__AAA__

In [71]:
print(*list(df_food_abc[df_food_abc['ABC']=='AAA']['article_name'].unique()), sep=', \n')

Дет. Хинкальки для малышей с мясом, 
Чебурек со свининой/говядиной, 
Соус чесночный, 
Хинкали с мраморной говядиной, 
4шт Хинкали с говядиной и свининой, 
4шт Хинкали с грибами, 
Хинкали с говядиной и свининой, 
4шт Хинкали с мраморной говядиной, 
Хинкали с грибами, 
Шампиньоны запеченные с сыром, 
Салат грузинский со специями, 
Микс хинкали 4шт, 
Салат с языком и гренками, 
Хинкали с бараниной и говядиной, 
Хачапури по-аджарски, 
4шт Хинкали с бараниной и говядиной


Это довольно многочисленная группа товаров с высоким объемом продаж, выручкой и прибылью. Задача: масштабировать успех. 

   - Цены: теоретически возможно немного поднять цены, так как эти товары пользуются большим спросом. По результатам `XYZ` анализа (проведенном в другом проекте) мы знаем, что эти позиции находятся в группе `X`, а именно обладают регулярным спросом. Если бы эти товары были в группе `Y`, `Z`, то лучше было бы вообще не поднимать цены (возможно, спрос в штуках был велик из-за акции на позиции) и сосредоточиться на том, чтобы переместить эти товары в группу `X`, а уже потом пытаться зарабатывать на них еще больше, чем сейчас. 
   - Реклама: Рекламировать для увеличения объема продаж не обязательно, так как они уже хорошо продаются. В нашем случае эти позиции являются самой сутью наших ресторанов, основным имиджевым (и не только) продуктом. 
   - Ассортимент: Необходимо обеспечить постоянное наличие этих товаров, чтобы предотвратить снижение продаж по причине невозможности удовлетворить весь спрос.

__AAB, AAC, ABA, ABB__

In [72]:
print(*list(df_food_abc[df_food_abc['ABC']=='AAB']['article_name'].unique()), sep=', \n')

Аджарик, 
Суп Харчо, 
Хачапури по-мегрельски, 
Цезарь по-восточному, 
Шашлык из свинины, 
Хинкали с сыром сулугуни


In [73]:
print(*list(df_food_abc[df_food_abc['ABC']=='AAC']['article_name'].unique()), sep=', \n')

4шт Хинкали с сыром, 
Шашлык из куриного бедра


In [74]:
print(*list(df_food_abc[df_food_abc['ABC']=='ABA']['article_name'].unique()), sep=', \n')

Грузинский лаваш пури, 
Сацебели, 
Сметана, 
Аджика красная, 
Чебурек с бараниной/говядиной


In [75]:
print(*list(df_food_abc[df_food_abc['ABC']=='ABB']['article_name'].unique()), sep=', \n')

Дет. Картофель фри


Это позиции, которые хорошо продаются (в штуках), но имеют либо среднюю выручку, либо среднюю / низкую доходность. В нашем случае для `AAB` и `AAC` это происходит потому, что себестоимость их достаточно высока из-за продуктов, входящих в их состав, а для `ABA` из-за их невысокой цены (при небольшой себестоимости) и того, что они хорошо продаются потому что являются популярными спутниками хинкали, товара из группы `AAA`.

Для `AAB` и `AAC`: возможно сделать специальные предложения (например, комбо-наборы с этими блюдами в сочетании с блюдами, имеющими высокую маржинальность, например, из `AAA` или `CCA`). Комбо-набор может быть дешевле, чем суммарный заказ этих блюд по-отдельности, при этом бОльшая маржинальность второго товара сможет компенсировать скидку и повысить средний чек.

Для `AAB`, `ABB` и `AAC`: увеличить доходность за счет понижения себестоимости.

### группа B**

Эти блюда менее популярны, но при этом не игнорируются, и также приносят довольно значительную часть дохода. Позиции из данной группы рекомендуется "перетянуть" в группу `А`.

Маркетинговые мероприятия:
1. Проведение рекламных кампаний для привлечения внимания к этим блюдам с использованием зрелищных и привлекательных методов, чтобы увеличить спрос на них.
2. Создание программ лояльности и специальных предложений для стимулирования повторной покупки данных блюд и увеличения среднего чека.
3. Использование тактики кросс-продаж, предлагая дополнительные товары или услуги в связке с данными блюдами для увеличения среднего чека.

Товарные мероприятия:
1. Анализ ценовой политики, для максимизации прибыли с каждой продажи данных блюд без ущерба для их конкурентоспособности и спроса.
2. Оптимизация процессов производства и поставок для эффективного управления запасами и снижения издержек производства.
3. Развитие идеи комбо-предложений, позволяющих клиентам сэкономить на заказе данных блюд, что должно повысить их привлекательность.

In [76]:
print(*list(df_food_abc[df_food_abc['ABC']=='BAA']['article_name'].unique()), sep=', \n')

Чебуреки ассорти, 
Ассорти рулетиков с сыром надуги с мятой, 
Запеченный сыр сулугуни с овощами, 
Салат с баклажаном и сыром фета


`ВАА`- увеличить продажи, рекламируя товар, прививая любовь к блюду

In [77]:
print(*list(df_food_abc[df_food_abc['ABC']=='BAB']['article_name'].unique()), sep=', \n')

Хачапури по-имеретински, 
Оджахури со свининой, 
Хачапургер, 
Салат Бабушки Нино, 
Долма с бараниной и говядиной, 
Ассорти закусок "Кутаиси", 
Мясное ассорти "Садж", 
Шашлык из индейки, 
Чашушули с говядиной


In [78]:
print(*list(df_food_abc[df_food_abc['ABC']=='BBA']['article_name'].unique()), sep=', \n')

Картофель по-домашнему, 
Пхали ассорти, 
Ассорти пхали на лаваше, 
Куриный бульон с пшеничной лапшой, 
Дет. Хинкальки для малышей с сыром, 
Мини-хинкали в бульоне, 
Овощи на гриле, 
Чебурек с сыром и зеленью, 
Салат с помидорами и красным луком


`ВАВ`, `BAB` - увеличить продажи и привить любовь к блюду (добавить в комбо-наборы с товарами `ААА`), чтобы перевести в группу `А`, увеличить доходность за счет понижения себестоимости

In [79]:
print(*list(df_food_abc[df_food_abc['ABC']=='BBB']['article_name'].unique()), sep=', \n')

Чахохбили с цыпленком, 
Шаверма по-батумски


`ВВB`  - увеличить продажи и привить любовь к блюду (добавить в комбо-наборы с товарами `ААА`), чтобы перевести в группу `А`

In [80]:
print(*list(df_food_abc[df_food_abc['ABC']=='BBC']['article_name'].unique()), sep=', \n')

Пицца Рача по-чкмерски, 
Дет. Пиццури с сыром и ветчиной, 
Суп Хашлама с бараниной и овощами


`ВBC`- увеличить доходность за счет понижения себестоимости

__BCA, BCC__

В данном случае получившиеся позиции в `BCA`, `BCC` - продолжение списка позиций в `CCA` (см.ниже) и рекомендации по ним те же. Для топпингов всегда предлагать для блюд `AAA`, класть в подарок на доставку и самовывоз, чтобы клиенты пробовали вкусные соусы и заказывали в следующий раз.

In [81]:
print(*list(df_food_abc[df_food_abc['ABC']=='BCA']['article_name'].unique()), sep=', \n')

Дет. Куриный бульон с алфавитом, 
Баже


In [82]:
print(*list(df_food_abc[df_food_abc['ABC']=='BCC']['article_name'].unique()), sep=', \n')

Ткемали зеленый, 
Ткемали красный, 
Дет. Куриный шашлычок, 
Наршараб


### группа C**

Теоретичеки эти блюда не пользуются большим успехом у гостей, поэтому приносят заведению меньшую долю прибыли. Причиной этому может быть неправильное соотношение цена-качество, оказавшееся не в пользу гостя или неверное позиционирование. 

Однако может быть такая ситуация, что эти товары находятся в группе `C` несправедливо, а именно плохо продаются не потому, что они не интересны, а потому что их просто нет в наличии тогда, когда на них есть спрос. Поэтому прежде, чем делать выводы по этим товарам, нужно проверить, что они доступны для покупки всегда, когда есть спрос.

__CBA__

In [83]:
print(*list(df_food_abc[df_food_abc['ABC']=='CBA']['article_name'].unique()), sep=', \n')

Аджабсандал, 
Оджахури с треской, 
Капрезе по-восточному


`СВА` - высокая цена, мало продаж в штуках, хорошая доходность - продвигать блюда, рекламировать как флагманы (тянуть в `ВВА`)

__CBB__

In [84]:
print(*list(df_food_abc[df_food_abc['ABC']=='CBB']['article_name'].unique()), sep=', \n')

Треска на подушке из овощей, 
Пицца Батуми с томатами и сулугуни, 
Шашлык из баранины, 
Сациви с курой


Это мало продающиеся позиции, имеющие среднюю доходность и высокую или среднюю маржинальность. Все это - позиции основного меню. Цель: переместиться в `BBA` (что затруднительно из-за высокой себестоимости входящих в состав компонентов) или хотя бы `BBB`. Поэтому необходимо продвигать блюда, рекламировать как аутентичные блюда-флагманы. 

__ССA__

Самая многочисленная у нас группа товаров: с низкими показателями по объему продаж и выручке и высоким показателем по прибыльности. 

In [85]:
print(*list(df_food_abc[df_food_abc['ABC']=='CCA']['article_name'].unique()), sep=', \n')

Каша овсяная на воде, 
Пхали со свеклой, 
Мини-хинкали с аджабсандалом, 
4 пхали на лаваше со свеклой, 
Огурцы св 100 гр, 
Пхали на лаваше со свеклой, 
Дет. Макарошки, 
Хинкали с яблоком и вишней, 
Капуста по-гурийски, 
Соевый соус 30 гр, 
Кинза 20 гр, 
Зелень свежая, 
Чвиштари, 
Петрушка 20 гр, 
Перец чили красный 10 гр, 
Каша овсяная на молоке, 
4 пхали на лаваше с баклажаном, 
Пхали с баклажаном , 
сп. Аджика сухая 10 гр, 
Пхали с грибами, 
Огурцы соленые 50 гр, 
Майонез 50 гр, 
Сацебели зеленый, 
Пхали на лаваше с баклажаном, 
Яйца отварные, 
Сезонные свежие овощи, 
Хачапури на шампуре, 
4 пхали на лаваше со шпинатом, 
сп. Перец чили сухой 10 гр, 
Джонджоли 50 гр, 
Пхали со шпинатом, 
Пхали на лаваше со шпинатом, 
Картофельное пюре, 
Сыр сулугуни 50 гр, 
Уксус винный 30 гр, 
Пхали на лаваше с грибами, 
Свежие овощи, 
4 пхали на лаваше с грибами, 
Яйцо куриное, 
Суп Бозбаш, 
Дет. Омлет Цыпа, 
Пицца Мцхета с грушей и дорблю, 
Сыр на хачапури 50 гр, 
Дет. Овощные палочки, 
Омлет, 
Люл

Здесь у нас топпинги и соусы, позиции Детского меню и несколько позиций основного меню. 

В нашем случае рекомендуется оставить без изменений топпинги и соусы (возможно, вывести те, что продаются по 1-5 штук за период), пока не менять позиции Детского меню, т.к. продажи по нему вообще не предполагаются значительными, скорее, оно нужно для имиджа (по брендбуку). 

По завтракам (каша, яичницы, омлеты и т.д.) - можно сократить ассортимент яичниц, например. Тогда из трех яичниц из `CCA` мы можем получить две позиции из `BBA`. Хотя можно этого и не делать, поскольку никакие ингредиенты, использующиеся при приготовлении яичницы или омлета, не держатся в ресторане специально, все они участвуют в других блюдах, поэтому необходимость поддерживать наличие яичниц в меню не требует дополнительной заморозки средств в этих ингредиентах. То, что готовится для завтраков, также не требует участия заготовительного цеха, поэтому скорее рекомендуется провести акции на завтраки, что также может помочь решить вопрос пониженной загрузки ресторана в утренние и дневные часы.

По позициям основного меню (особенно пхали, т.к. это одна из основных имиджевых позиций меню, и отказаться от неё нельзя) цель такая: переместиться в `BBA`. Поэтому общие рекомендации такие: 
- Цены: Можно рассмотреть возможность снижения цен, но только если это не ухудшит общую прибыль.
- Реклама: рекламировать с особым акцентом на их качество или уникальность как аутентичного национального блюда. 
- Ассортимент: Рассмотреть возможность сокращения ассортимента пхали, оставив наиболее популярные их варианты. Также, возможно, клиенты могли не пробовать пхали никогда вообще, поэтому их можно приносить в "счастливые" часы как комплимент от заведения при покупке определенных позиций из барного меню. 

__ССB__

In [86]:
print(*list(df_food_abc[df_food_abc['ABC']=='CCB']['article_name'].unique()), sep=', \n')

Дет. Сосиска в тесте, 
Пицца Кахетия с аджабсандалом, 
Язык отварной 50 гр, 
Масло оливковое 30 гр, 
Соленья, 
Помидоры соленые 50 гр, 
Хачапури по-гурийски с ветчиной, 
Томаты свежие, 
Чакапули, 
Дет. Суп с фрикадельками, 
Дет. Сэндвич Совушка, 
Мясное ассорти, 
Зерна граната 20 гр, 
Вишневый соус 50гр


Для топпингов и соусов рекомендуется всегда предлагать их при заказе хинкали и салатов (в ресторане: официантами, принимающими заказ, при заказе на сайте или в приложении: через всплывающие рекомендации или рекомендации в корзине).

__CCC__

In [87]:
print(*list(df_food_abc[df_food_abc['ABC']=='CCC']['article_name'].unique()), sep=', \n')

Сыр чечил копченый 30 гр, 
Огурцы маринованные 50 гр, 
Дет. Куриная котлета, 
Дет. Наггетсы рыбные в кляре, 
Сыр топпинг, 
Ветчина топпинг, 
Бастурма 30 гр, 
Доставка 3-я зона , 
Соус табаско 10 гр, 
Орех грецкий топпинг


Группа товаров с низкими показателями по объему продаж, выручке и прибыли (код "CCC"): 
- здесь это топпинги и добавки и позиции Детского меню. 

Топпинги помогают повышать средний чек, а позиции Детского меню необходимы для бренд-бука, поэтому выводить их не стоит в данном случае, хотя вообще по этой группе дается рекомендация о рассмотрении варианта вывода позиций (особенно для детского меню, т.к. котлеты и наггетсы закупаются отдельно и имеют срок хранения, и при невостребованности списываются).

__CCB__

In [88]:
print(*list(df_food_abc[df_food_abc['ABC']=='CCB']['article_name'].unique()), sep=', \n')

Дет. Сосиска в тесте, 
Пицца Кахетия с аджабсандалом, 
Язык отварной 50 гр, 
Масло оливковое 30 гр, 
Соленья, 
Помидоры соленые 50 гр, 
Хачапури по-гурийски с ветчиной, 
Томаты свежие, 
Чакапули, 
Дет. Суп с фрикадельками, 
Дет. Сэндвич Совушка, 
Мясное ассорти, 
Зерна граната 20 гр, 
Вишневый соус 50гр


Группа товаров с низкими показателями по объему продаж и выручке и средним показателем по прибыли: 
- здесь топпинги и добавки, позиции Детского меню и уже выведенная из основного меню пицца.

__CBC__

Группа товаров с низкими показателями по объему продаж, средним показателем по выручке и низким показателем по прибыли. Это дорогие позиции с высокой себестоимостью, которые, тем не менее, являются имиджевыми в меню и не могут быть выведены из него. 

In [89]:
print(*list(df_food_abc[df_food_abc['ABC']=='CBC']['article_name'].unique()), sep=', \n')

Баранья лопатка на кости, 
Пицца Сванети мясная, 
Ассорти сыров


Рекомендация: продвигать позиции как флагманские с целью перейти в `BBС`. Также, по возможности, попробовать снизить себестоимость (но для данных конкретных позиций это малоперспективно, т.к. может пострадать качество).

__Для получения более глубокой аналитики__ рекомендуется провести `ABC-анализ` в следующих разрезах:
- по бренду / ресторану,
- по типу заказа (доставка, самовывоз, алякарт)
- по группе меню (детское, основное, сезонное, барное, чайное и т.д.)